# Final Project
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__  
Throughout this course you’ve engaged with key principles required to develop scalable machine learning analyses for structured and unstructured data. Working in Hadoop Streaming and Spark you’ve learned to translate common machine learning algorithms into Map-Reduce style implementations. You’ve developed the ability to evaluate Machine Learning approaches both in terms of their predictive performance as well as their scalability. For the final project you will demonstrate these skills by solving a machine learning challenge on a new dataset. Your job is to perform Click Through Rate prediction on a large dataset of Criteo advertising data made public as part of a Kaggle competition a few years back. As you perform your analysis, keep in mind that we are not grading you on the final performance of your model or how ‘advanced’ the techniques you use but rather on your ability to explain and develop a scalable machine learning approach to answering a real question.

More about the dataset:
https://www.kaggle.com/c/criteo-display-ad-challenge

# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [57]:
import re
import ast
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import col,sum,when,isnan,count
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session
app_name = "fproj_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [5]:
# create a data directory (RUN THIS CELL AS IS)
# !mkdir data

In [6]:
# grab the tar.gz file from kaggle

# !curl https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz -o data/dac.tar.gz

In [7]:
# I couldn't get this to unpack the tarball, so I just did it in Windows
#!tar -xvz data/dac.tar.gz

In [8]:
# take a look at the to the training data set
# !head data/train.txt

In [9]:
# ! ls ../Assignments/FinalProject/

In [5]:
# load the data into Spark RDD for convenience of use later (RUN THIS CELL AS IS)
projectRDD = sc.textFile('../Assignments/FinalProject/data/train.txt')
sampleRDD = sc.textFile('../Assignments/FinalProject/data/train1-500.txt')

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

# Question 1: Question Formulation 
Introduce the goal of your analysis. What questions will you seek to answer, why do people perform this kind of analysis on this kind of data? Preview what level of performance your model would need to achieve to be practically useful.

# Question 2: Algorithm Explanation
Create your own toy example that matches the dataset provided and use this toy example to explain the math behind the algorithym that you will perform.

# Question 3: EDA & Discussion of Challenges
Determine 2-3 relevant EDA tasks that will help you make decisions about how you implement the algorithm to be scalable. Discuss any challenges that you anticipate based on the EDA you perform

In [7]:
sqlContext = SQLContext(sc)

# Load a text file and convert each line to a Row.
# lines = sc.textFile("../Assignments/FinalProject/data/train.txt")

lines=sampleRDD
parts = lines.map(lambda l: l.split("\t"))
project = parts.map(lambda p: Row(label=int(p[0]), I1=p[1], I2=p[2],\
                    I3=p[3], I4=p[4], I5=p[5], I6=p[6],\
                    I7=p[7], I8=p[8], I9=p[9], I10=p[10],\
                    I11=p[11], I12=p[12], I13=p[13], C1=p[14], C2=p[15], C3=p[16],\
                    C4=p[17], C5=p[18], C6=p[19], C7=p[20], C8=p[21], C9=p[22],\
                    C10=p[23], C11=p[24], C12=p[25], C13=p[26], C14=p[27], C15=p[28],\
                    C16=p[29], C17=p[30], C18=p[31], C19=p[32], C20=p[33], C21=p[34],\
                    C22=p[35], C23=p[36], C24=p[37], C25=p[38], C26=p[39]))

# Infer the schema, and register the DataFrame as a table.
projectDF = sqlContext.createDataFrame(project)
projectDF.registerTempTable("projectTable")


In [8]:
projectDF.count()

500

In [13]:
projectDF.printSchema()

root
 |-- C1: string (nullable = true)
 |-- C10: string (nullable = true)
 |-- C11: string (nullable = true)
 |-- C12: string (nullable = true)
 |-- C13: string (nullable = true)
 |-- C14: string (nullable = true)
 |-- C15: string (nullable = true)
 |-- C16: string (nullable = true)
 |-- C17: string (nullable = true)
 |-- C18: string (nullable = true)
 |-- C19: string (nullable = true)
 |-- C2: string (nullable = true)
 |-- C20: string (nullable = true)
 |-- C21: string (nullable = true)
 |-- C22: string (nullable = true)
 |-- C23: string (nullable = true)
 |-- C24: string (nullable = true)
 |-- C25: string (nullable = true)
 |-- C26: string (nullable = true)
 |-- C3: string (nullable = true)
 |-- C4: string (nullable = true)
 |-- C5: string (nullable = true)
 |-- C6: string (nullable = true)
 |-- C7: string (nullable = true)
 |-- C8: string (nullable = true)
 |-- C9: string (nullable = true)
 |-- I1: string (nullable = true)
 |-- I10: string (nullable = true)
 |-- I11: string (nullabl

In [9]:
projectDF = projectDF.withColumn("I1", projectDF["I1"].cast("int"))
projectDF = projectDF.withColumn("I2", projectDF["I2"].cast("int"))
projectDF = projectDF.withColumn("I3", projectDF["I3"].cast("int"))
projectDF = projectDF.withColumn("I4", projectDF["I4"].cast("int"))
projectDF = projectDF.withColumn("I5", projectDF["I5"].cast("int"))
projectDF = projectDF.withColumn("I6", projectDF["I6"].cast("int"))
projectDF = projectDF.withColumn("I7", projectDF["I7"].cast("int"))
projectDF = projectDF.withColumn("I8", projectDF["I8"].cast("int"))
projectDF = projectDF.withColumn("I9", projectDF["I9"].cast("int"))
projectDF = projectDF.withColumn("I10", projectDF["I10"].cast("int"))
projectDF = projectDF.withColumn("I11", projectDF["I11"].cast("int"))
projectDF = projectDF.withColumn("I12", projectDF["I12"].cast("int"))
projectDF = projectDF.withColumn("I13", projectDF["I13"].cast("int"))

In [15]:
projectDF.printSchema()

root
 |-- C1: string (nullable = true)
 |-- C10: string (nullable = true)
 |-- C11: string (nullable = true)
 |-- C12: string (nullable = true)
 |-- C13: string (nullable = true)
 |-- C14: string (nullable = true)
 |-- C15: string (nullable = true)
 |-- C16: string (nullable = true)
 |-- C17: string (nullable = true)
 |-- C18: string (nullable = true)
 |-- C19: string (nullable = true)
 |-- C2: string (nullable = true)
 |-- C20: string (nullable = true)
 |-- C21: string (nullable = true)
 |-- C22: string (nullable = true)
 |-- C23: string (nullable = true)
 |-- C24: string (nullable = true)
 |-- C25: string (nullable = true)
 |-- C26: string (nullable = true)
 |-- C3: string (nullable = true)
 |-- C4: string (nullable = true)
 |-- C5: string (nullable = true)
 |-- C6: string (nullable = true)
 |-- C7: string (nullable = true)
 |-- C8: string (nullable = true)
 |-- C9: string (nullable = true)
 |-- I1: integer (nullable = true)
 |-- I10: integer (nullable = true)
 |-- I11: integer (null

In [16]:
# numRows = projectRDD.count()
# numRows

In [17]:
# # get a sample to do things at a small scale with
# sampleSize = 10000 # change this as we get sense of compute times
# sampleDF = projectDF.sample(withReplacement = False, fraction = sampleSize/numRows, seed = 1000) 
# sampleDF.createOrReplaceTempView("sampleDFTable") # I don't really know what this does

In [10]:
def handle_missing(data):
    """
    Replaces missiong values ('',' ',NA, NaN) with specified value
    Args:
        data - Spark DF with missing values
        filler - Value to replace missing with defaults to 0
    Returns:
        data - Spark DF with missing vales filled in
    """
    
    for c in data.columns:
        data = data.replace(' ',str('EMPTY'), c)
        data = data.replace('',str('EMPTY'), c)
    data = data.na.fill(0)
    return data

def pipeline_stages(categorical_features,numeric_features):
    """Constructs a list of pipeline stages which will transform
       All categorical features into one-hot vectors
       
       Args:
           categorical_features - list of categorical feature column names
           numeric_features - list of numeric feature column names
       Returns: 
           stages - list of pipeline stages which will convert categorical features to
               one-hot vectors
        """
    stages = []
    for cat_col in categorical_features:
        stringIndexer = StringIndexer(inputCol = cat_col, outputCol = cat_col + 'Index')
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[cat_col + "classVec"],handleInvalid='keep')
        stages += [stringIndexer, encoder]

    # label_stringIdx = StringIndexer(inputCol = 'label', outputCol = 'label')
    # stages += [label_stringIdx]

    assemblerInputs = [c + "classVec" for c in categorical_features] + numeric_features
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    stages += [assembler]
    
    return stages
def apply_pipeline(data,stages):
    """
    Applies pipeline stages to data
    Args: 
        data - Spark DF of data to be modified
        stages - list of pipeline stages to be applied to data
    returns:
        preppedDataDF - Spark DF with variable transformations applied

    """
    partialPipeline = Pipeline().setStages(stages)
    pipelineModel = partialPipeline.fit(data)
    preppedDataDF = pipelineModel.transform(data)

    
    return preppedDataDF

In [11]:
numeric_features = ["I1", "I2", "I3", "I4", "I5", "I6", "I7", "I8", "I9", "I10", "I11", "I12", "I13"]

In [12]:
categorical_features = ["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", "C11", "C12",\
                        "C13", "C14", "C15", "C16", "C17", "C18", "C19", "C20", "C21", "C22", "C23",\
                       "C24", "C25", "C26"]

In [13]:
label_and_numeric = ["label", "I1", "I2", "I3", "I4", "I5", "I6", "I7", "I8", "I9", "I10", "I11", "I12", "I13"]

In [14]:
weights = [.8, .2]
seed = 4
train,test = projectDF.randomSplit(weights,seed)

train = handle_missing(train)
test = handle_missing(test)

stages = []
stages = pipeline_stages(categorical_features,numeric_features)

for c in train.columns:
    train = train.replace(' ',str('EMPTY'), c)
    train = train.replace('',str('EMPTY'), c)
train = train.na.fill(0)

In [45]:
preppedTrainDF = apply_pipeline(train,stages).cache()
preppedTestDF = apply_pipeline(test,stages).cache()

# preppedTrainDF = preppedTrainDF.na.fill(0)
# preppedTestDF = preppedTestDF.na.fill(0)

lrModel = LogisticRegression().fit(preppedTrainDF)
# display(lrModel, preppedDataDF, "ROC")

trainingSummary = lrModel.summary
roc = trainingSummary.roc.toPandas()

predictions = lrModel.transform(preppedTestDF)

In [81]:
type(preppedTestDF.toPandas().features.iloc[0])

pyspark.ml.linalg.SparseVector

In [85]:
 len(str(lrModel.coefficients))
# print("Intercept: " + str(lrModel.intercept))

71061

In [68]:
predictions = lrModel.transform(preppedTestDF)
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
# evaluator.evaluate(predictions)

# Question 4: Algorithm Implementation 
Develop a 'homegrown' implementation of the algorithn, apply it to the training dataset and evaluate your results on the test set. 

# Question 5: Application of Course Concepts
Pick 3-5 key course concepts and discuss how your work on this assignment illustrates an understanding of these concepts. 

In [69]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(preppedTrainDF)

In [70]:
display(dtModel)
predictions = dtModel.transform(preppedTestDF)
evaluator = BinaryClassificationEvaluator()

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4063ad08dc434139cd67) of depth 3 with 13 nodes